# 서울시 휴게음식점, 인구수 데이터 Open API 로 불러오기
서울시 인구 데이터: http://data.seoul.go.kr/dataList/OA-15379/S/1/datasetView.do
  
서울시 휴게음식점 데이터: https://data.seoul.go.kr/dataList/OA-16095/S/1/datasetView.do 

In [1]:
import pandas as pd
import requests

In [2]:
# 서울시 열린데이터에서 서비스명을 구할 수 있는 함수입니다.
def getdataname(url):
    title = pd.read_html(url, encoding="utf8")
    return title[1].iloc[2][3]

In [3]:
# 서울시 열린데이터에서 컬럼명을 구할 수 있는 함수입니다.
def getcolumnname(url):
    title = pd.read_html(url, encoding="utf8")
    return title[2].iloc[3:]["출력설명"].values.tolist()

In [8]:
# 서울시 열린데이터에서 open API를 이용하여 데이터를 불러오는 함수입니다.
def loaddata(url):
    # 반복문을 위한 변수를 먼저 지정해줍니다.
    page = 1
    list = []
    apikey = '704b7877666a687339376a75504856'
    dataname = getdataname(url)

    # json을 반복적으로 불러 올 수 있게 반복문을 이용합니다.
    while True:

        # open API 주소를 이용합니다. json으로 받아옵니다. 최대로 가져올 수 있는 데이터가 1000개라 데이터의 갯수가 1000개가 되도록 변수를 지정합니다.
        apiurl = f'http://openapi.seoul.go.kr:8088/{apikey}/json/{dataname}/{page}/{page+999}/'

        # requests를 이용하여 json을 불러옵니다.
        response = requests.get(apiurl).json()
        temp = response[dataname]['row']

        # page마다 나오는 results 값들을 list에 합쳐줍니다.
        list = list + temp

        # 최대로 가져올 수 있는 데이터가 1000개라 천씩 높혀주어 계속해서 json을 불러올 수 있게합니다.
        page = page + 1000

        # page 전체 개수보다 작게 해서 반복문 탈출함
        if page > response[dataname]['list_total_count']:
            break

    # DataFrame으로 변환합니다.
    df = pd.DataFrame(list)
    df.columns = getcolumnname(url)
    return df

In [5]:
# 개발자 도구로 구한 url
storedataurl = "https://data.seoul.go.kr/dataList/openApiView.do?infId=OA-16095&srvType=A&isLogin=&srvType=S&infId=OA-16095&serviceKind=1&ssUserId=&infNm=%EC%84%9C%EC%9A%B8%EC%8B%9C+%ED%9C%B4%EA%B2%8C%EC%9D%8C%EC%8B%9D%EC%A0%90+%EC%9D%B8%ED%97%88%EA%B0%80+%EC%A0%95%EB%B3%B4&stcRow="
populationdataurl = "https://data.seoul.go.kr/dataList/openApiView.do?infId=OA-15379&srvType=A&isLogin=&srvType=S&infId=OA-15379&serviceKind=1&ssUserId=&infNm=%EC%9E%90%EC%B9%98%EA%B5%AC%EB%8B%A8%EC%9C%84+%EC%84%9C%EC%9A%B8%EC%83%9D%ED%99%9C%EC%9D%B8%EA%B5%AC+%EC%9D%BC%EB%B3%84+%EC%A7%91%EA%B3%84%ED%91%9C&stcRow="

In [9]:
storedf = loaddata(storedataurl)
print(storedf.shape)
populationdf = loaddata(populationdataurl)
print(populationdf.shape)

(120208, 44)
(42640, 15)


In [10]:
# 모든 컬럼이 출력되게 설정합니다.
pd.options.display.max_columns = None

In [11]:
storedf

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,휴업종료일자,재개업일자,전화번호,소재지면적,소재지우편번호,지번주소,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),위생업태명,남성종사자수,여성종사자수,영업장주변구분명,등급구분명,급수시설구분명,총인원,본사종업원수,공장사무직종업원수,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3130000,3130000-104-2022-00101,20220422,,01,영업/정상,01,영업,,,,,,89.25,121849,서울특별시 마포구 성산동 515 월드컵주경기장,"서울특별시 마포구 월드컵로 240, 홈플러스 월드컵점 2층 x9.5,yuw호 (성산동)",03932,팜스365,20220422114307,I,2021-12-03 22:04:00.0,커피숍,191010.381048042,451972.154018501,,,,,,,,,,,,,,,,,,,
1,3230000,3230000-104-2022-00095,20220422,,01,영업/정상,01,영업,,,,,,22.96,138960,서울특별시 송파구 문정동 634 가든파이브라이프,"서울특별시 송파구 충민로 66, 가든파이브라이프 6층 T-6131호 (문정동)",05838,패니발코리아(Fanival Korea),20220422131317,I,2021-12-03 22:04:00.0,기타 휴게음식점,210986.460698452,441725.293491662,,,,,,,,,,,,,,,,,,,
2,3230000,3230000-104-2022-00096,20220422,,01,영업/정상,01,영업,,,,,,3.30,138829,서울특별시 송파구 방이동 89 올림픽선수기자촌아파트,"서울특별시 송파구 양재대로 1218, F동 1층 106호 (방이동, 올림픽선수기자촌...",05649,씨유 송파올림픽점,20220422172210,I,2021-12-03 22:04:00.0,편의점,211986.297245596,446156.529775445,,,,,,,,,,,,,,,,,,,
3,3180000,3180000-104-2022-00128,20220628,,01,영업/정상,01,영업,,,,,,11.34,150836,서울특별시 영등포구 문래동3가 82-3,"서울특별시 영등포구 문래북로 54, 1층 (문래동3가)",07294,"그리고, 커피",20220628142751,I,2021-12-05 21:00:00.0,커피숍,190310.553874627,446496.894115624,,,,,,,,,,,,,,,,,,,
4,3120000,3120000-104-2022-00060,20220422,,01,영업/정상,01,영업,,,,,,231.01,120807,서울특별시 서대문구 남가좌동 343-4 1-2층,"서울특별시 서대문구 거북골로 41-1, 1,2층 (남가좌동)",03665,할리스 서울명지대점,20220422114050,I,2021-12-03 22:04:00.0,커피숍,193157.276918756,452964.783159586,,,,,,,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120203,3130000,3130000-104-2014-00165,20140714,,01,영업/정상,01,영업,,,,,0232111119,6.60,121130,서울특별시 마포구 구수동 68-8 1층일부,"서울특별시 마포구 토정로 192 (구수동, 1층일부)",04093,지에스(GS)25 서강자이점,20220513112752,U,2021-12-04 23:05:00.0,커피숍,193938.78595938,449222.448051881,,,,,,,,,,,,,,,,,,,
120204,3050000,3050000-104-2011-00066,20110726,,03,폐업,02,폐업,20220513,,,,02 7526252,210.60,130813,서울특별시 동대문구 신설동 109-4 지하1층,"서울특별시 동대문구 천호대로4길 22, 지하1층 (신설동)",02586,(주)다복캐터링 동대문점,20220513153049,U,2021-12-04 23:05:00.0,일반조리판매,202088.522253764,452350.834476363,,,,,,,,,,,,,,,,,,,
120205,3220000,3220000-104-1997-01153,19971231,,03,폐업,02,폐업,20220513,,,,02 5153642,32.40,135898,서울특별시 강남구 압구정동 369-1 구상가C동 지하동 4호호,"서울특별시 강남구 압구정로29길 71, 지하동 4호호 (압구정동,구상가C동)",06002,블루,20220513132121,U,2021-12-04 23:05:00.0,다방,202398.594377612,447864.763737276,,,,,,,,,,,,,,,,,,,
120206,3210000,3210000-104-2022-00116,20220420,,03,폐업,02,폐업,20220513,,,,031 268 5468,.00,137713,서울특별시 서초구 반포동 19-3 신세계백화점 강남점 지하1층,"서울특별시 서초구 신반포로 176, 신세계백화점 강남점 지하1층 (반포동)",06546,(주)제이와이에스유통,20220514041509,U,2021-12-04 23:07:00.0,기타 휴게음식점,200250.447804795,444683.220506107,,,,,,,,,,,,,,,,,,,


In [12]:
populationdf

,기준일ID,시군구코드,시군구명,총생활인구수,내국인생활인구수,장기체류외국인인구수,단기체류외국인인구수,일최대인구수,일최소인구수,주간인구수(09~18),야간인구수(19~08),일최대이동인구수,서울외유입인구수,동일자치구행정동간이동인구수,자치구간이동인구수
0,20180416,11200,성동구,378037.34560,359785.27660,16452.80070,1799.26820,394446.09320,357331.94590,391622.81380,368333.43970,246418.69350,44411.50070,79693.05870,122314.13410
1,20180422,11320,도봉구,315300.31120,311748.74150,3130.38200,421.18770,329305.07040,299838.01020,306446.52530,321624.44400,148838.05940,21785.10880,83432.48070,43620.46990
2,20180425,11170,용산구,337378.29440,309661.71770,18552.34510,9164.23150,374936.98390,290607.00890,371318.43270,313135.33850,235926.58620,61661.12900,55248.42630,119017.03090
3,20180415,11470,양천구,427605.74270,421005.44800,5413.90690,1186.38790,445511.05770,401617.16050,412527.62710,438375.82530,203915.30540,31644.99010,107993.82950,64276.48580
4,20180413,11110,종로구,402328.92190,365087.40260,20389.49370,16852.02560,566803.38300,269438.95380,521132.43820,317469.26740,383003.77610,97579.74820,43663.90600,241760.12190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42635,20221013,11110,종로구,339717.78380,309660.00920,17834.93590,12222.83870,478788.97770,236988.38430,438635.59090,269062.20730,305940.99970,89544.54060,31340.38230,185056.07680
42636,20221013,11170,용산구,312022.60310,287441.09880,16348.87620,8232.62810,340408.88970,277739.23640,336051.45180,294859.13980,197229.06550,56143.17110,45285.76480,95800.12960
42637,20221013,11215,광진구,378560.57870,353736.18030,21410.38120,3414.01720,397986.71010,353111.02590,360059.88380,391775.36080,178914.95560,36716.13830,74879.81400,67319.00330
42638,20221013,11470,양천구,359794.01540,353962.31610,4688.90650,1142.79280,391417.16280,317053.69060,329281.79390,381588.45940,146965.16450,25514.39690,67204.45810,54246.30950
